<a href="https://colab.research.google.com/github/dolmani38/NOTAM/blob/main/albert_notam_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Albert NOTAM 언어 모델 학습!

In [1]:

if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


## 참조

https://colab.research.google.com/github/parmarsuraj99/suraj-parmar/blob/master/_notebooks/2020-05-02-SanskritALBERT.ipynb#scrollTo=VNAOMXjpMHZD


In [2]:
import os
import gc
import glob
import torch
import pickle
import joblib
from tqdm.auto import tqdm

In [3]:
!git clone https://github.com/huggingface/transformers
!pip install transformers/.
!pip install sentencepiece==0.1.95
!pip install datasets==1.8.0

Cloning into 'transformers'...
remote: Enumerating objects: 80215, done.
remote: Counting objects: 100% (1086/1086), done.
remote: Compressing objects: 100% (565/565), done.
remote: Total 80215 (delta 652), reused 777 (delta 452), pack-reused 79129
Receiving objects: 100% (80215/80215), 63.35 MiB | 28.34 MiB/s, done.
Resolving deltas: 100% (57253/57253), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |███████████████████████████

# NOTAM Dataset load

In [4]:
%%time

import re
import pandas as pd
import sys

df = pd.read_csv('/content/drive/MyDrive/NOTAM/data/TRAIN_210812.csv')


CPU times: user 27.4 ms, sys: 12 ms, total: 39.4 ms
Wall time: 533 ms


In [5]:
df

,MSG_ID,UPDATED_BY,UPDATED_DATE,E_SCORE,AE_SCORE,ALL_SCORE,Q_LINE,A_LINE,B_LINE,C_LINE,D_LINE,E_LINE
0,6824038,shbl1021,2021-06-08 9:08,1,1,1,YUXX/QOAXX/IV/BO/E/000/999/2537S13421E005,YBBB/YMMM,1905221600,1906191600,NaN,AIS (XX)\nAUSTRALIAN DIGITAL AERONAUTICAL FLIG...
1,6824038,parkij,2021-07-01 8:16,0,0,0,YUXX/QOAXX/IV/BO/E/000/999/2537S13421E005,YBBB/YMMM,1905221600,1906191600,NaN,AIS (XX)\nAUSTRALIAN DIGITAL AERONAUTICAL FLIG...
2,6816851,shbl1021,2021-06-08 9:09,5,5,2,LECM/QRDCA/IV/BO /W /000/100/3635N00636W022,LECM,1905270600,1905301559,"27-28 0600-1900, 30 1201-1559",LED128 ACTIVATED. LOWER AND UPPER VERTICAL LIM...
3,6816851,parkij,2021-07-01 8:18,5,5,0,LECM/QRDCA/IV/BO /W /000/100/3635N00636W022,LECM,1905270600,1905301559,"27-28 0600-1900, 30 1201-1559",LED128 ACTIVATED. LOWER AND UPPER VERTICAL LIM...
4,6816333,shbl1021,2021-06-08 9:11,4,1,1,KZOB/QMXLC/IV/BO/A/000/999/4213N08321W005,KDTW,1905211704,1905211800,NaN,TWY K5 CLSD
...,...,...,...,...,...,...,...,...,...,...,...,...
10410,15966850,parkij,2021-08-01 10:52,4,4,4,KZNY/QMXLC/IV/BO/A/000/999/4038N07347W005,KJFK,1912311421,1912311600,NaN,TWY C BTN TWY C1 AND TWY V CLSD
10411,15947262,shbl1021,2021-08-10 16:06,1,1,1,LCCC/QWFLW/IV/M /W /210/230/3335N03202E133,LCCC,2001040820,2001051150,0820-1150,AIR REFUELLING WILL TAKE PLACE ALONG \nSUVAS-K...
10412,15962821,shbl1021,2021-08-10 16:48,5,4,4,KZAU/QMRLC/IV/NBO/A/000/999/4257N08754W005,KMKE,1912310929,2001012359,NaN,RWY 01L/19R CLSD
10413,15965583,shbl1021,2021-08-06 13:52,4,4,3,RJJJ/QMXLC/IV/M/A/000/999/3533N13947E005,RJTT,2001071500,2001072000,NaN,TWY H(BTN E AND W) H1 H2 R(BTN H AND G)-CLSD D...


In [6]:

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    #txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    #txt = txt.replace(',','')
    #txt = txt.replace('..','')
    #txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    #txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

In [7]:
notam_contents = []

for row in df.iterrows():
    doc_cont = str(row[1][11])
    notam_contents.append(clean_text(doc_cont.lower()))
    notam_contents.append(clean_text(str(row[1][6]).lower()))

In [8]:
len(notam_contents)

20830

In [10]:
notam_contents[3]

'yuxx/qoaxx/iv/bo/e/000/999/2537s13421e005'

In [11]:
f = open('/content/drive/MyDrive/NOTAM/train_tokenizer2.txt', 'r')
while True:
    line = f.readline()
    if not line: break
    notam_contents.append(clean_text(line.lower()))
f.close()

In [12]:
len(notam_contents)

37602

In [13]:
%%time
# subword 학습을 위해 문장만 따로 저장
with open('/content/drive/MyDrive/NOTAM/train_tokenizer.txt', 'w', encoding='utf-8') as f:
    for line in notam_contents:
        f.write(line+'\n')

f.close()

CPU times: user 16 ms, sys: 4.81 ms, total: 20.8 ms
Wall time: 34 ms


# Tokenizer 학습


In [16]:
%%time
import sentencepiece as spm
import os
# spm_train --input=data/train_tokenizer.txt  --model_prefix=sentencepiece/sp --vocab_size=32000 character_coverage=1.0 --model_type="unigram"

input_file = '/content/drive/MyDrive/NOTAM/train_tokenizer.txt'
vocab_size = 21770

sp_model_root='/content/drive/MyDrive/NOTAM/albert_tokenizer_model'
if not os.path.isdir(sp_model_root):
    os.mkdir(sp_model_root)
sp_model_name = 'spiece'
sp_model_path = os.path.join(sp_model_root, sp_model_name)
model_type = 'unigram'  # 학습할 모델 선택, unigram이 더 성능이 좋음'bpe'
character_coverage  = 1.0  # 전체를 cover 하기 위해, default=0.9995
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS],[UNK0],[UNK1],[UNK2],[UNK3],[UNK4],[UNK5],[UNK6],[UNK7],[UNK8],[UNK9],[unused0],[unused1],[unused2],[unused3],[unused4],[unused5],[unused6],[unused7],[unused8],[unused9],[unused10],[unused11],[unused12],[unused13],[unused14],[unused15],[unused16],[unused17],[unused18],[unused19],[unused20],[unused21],[unused22],[unused23],[unused24],[unused25],[unused26],[unused27],[unused28],[unused29],[unused30],[unused31],[unused32],[unused33],[unused34],[unused35],[unused36],[unused37],[unused38],[unused39],[unused40],[unused41],[unused42],[unused43],[unused44],[unused45],[unused46],[unused47],[unused48],[unused49],[unused50],[unused51],[unused52],[unused53],[unused54],[unused55],[unused56],[unused57],[unused58],[unused59],[unused60],[unused61],[unused62],[unused63],[unused64],[unused65],[unused66],[unused67],[unused68],[unused69],[unused70],[unused71],[unused72],[unused73],[unused74],[unused75],[unused76],[unused77],[unused78],[unused79],[unused80],[unused81],[unused82],[unused83],[unused84],[unused85],[unused86],[unused87],[unused88],[unused89],[unused90],[unused91],[unused92],[unused93],[unused94],[unused95],[unused96],[unused97],[unused98],[unused99]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s --character_coverage=%s'
cmd = input_argument%(input_file, sp_model_path, vocab_size,user_defined_symbols, model_type, character_coverage)

spm.SentencePieceTrainer.Train(cmd)
print('train done')

train done
CPU times: user 2min 9s, sys: 6.75 s, total: 2min 15s
Wall time: 2min 12s


In [17]:
## check
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(sp_model_path))

tokens = sp.encode_as_pieces("ais (xx) australian digital aeronautical flight information file edition 8 (ausdafif ed 8) withdrawn. replacement version australian digital aeronautical flight information file edition 8 version 2 (ausdafif ed 8 v2) available via ais-af intranet. mil use only.")
ids = sp.encode_as_ids("ais (xx) australian digital aeronautical flight information file edition 8 (ausdafif ed 8) withdrawn. replacement version australian digital aeronautical flight information file edition 8 version 2 (ausdafif ed 8 v2) available via ais-af intranet. mil use only.")

print(ids)
print(tokens)

tokens = sp.decode_pieces(tokens)
ids = sp.decode_ids(ids)

print(ids)
print(tokens)

[2960, 158, 325, 150, 934, 121, 6471, 6891, 4465, 287, 5526, 227, 537, 784, 2603, 727, 158, 722, 13632, 11285, 124, 156, 482, 1503, 123, 3055, 203, 124, 9543, 934, 121, 6471, 6891, 4465, 287, 5526, 227, 537, 784, 2603, 727, 124, 9543, 244, 158, 722, 13632, 11285, 124, 156, 727, 309, 762, 542, 230, 2960, 133, 3530, 16070, 1695, 123, 493, 295, 217, 123]
['▁ais', '▁(', 'xx', ')', '▁australia', 'n', '▁dig', 'ital', '▁aerona', 'u', 'tical', '▁flight', '▁information', '▁file', '▁edition', '▁8', '▁(', 'au', 'sda', 'fif', '▁', 'ed', '▁8)', '▁withdrawn', '.', '▁replacemen', 't', '▁', 'version', '▁australia', 'n', '▁dig', 'ital', '▁aerona', 'u', 'tical', '▁flight', '▁information', '▁file', '▁edition', '▁8', '▁', 'version', '▁2', '▁(', 'au', 'sda', 'fif', '▁', 'ed', '▁8', '▁v', '2)', '▁available', '▁via', '▁ais', '-', 'af', '▁intran', 'et', '.', '▁mil', '▁use', '▁only', '.']
ais (xx) australian digital aeronautical flight information file edition 8 (ausdafif ed 8) withdrawn. replacement version a

# 여기서부터 다시

In [18]:
## 1) define special tokens
user_defined_symbols = ['[BOS]','[EOS]','[UNK0]','[UNK1]','[UNK2]','[UNK3]','[UNK4]','[UNK5]','[UNK6]','[UNK7]','[UNK8]','[UNK9]']
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
user_defined_symbols = user_defined_symbols + unused_list

print(user_defined_symbols)

['[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused34]', '[unused35]', '[unused36]', '[unused37]', '[unused38]', '[unused39]', '[unused40]', '[unused41]', '[unused42]', '[unused43]', '[unused44]', '[unused45]', '[unused46]', '[unused47]', '[unused48]', '[unused49]', '[unused50]', '[unused51]', '[unused52]', '[unused53]', '[unused54]', '[unused55]', '[unused56]', '[unused57]', '[unused58]', '[unused59]', '[unused60]', '[unused61]', '[unused62]', '[unused6

In [19]:
from transformers import AlbertTokenizer, AlbertTokenizerFast

#'/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model'

albet_tokenizer_model = '/content/drive/MyDrive/NOTAM/albert_tokenizer_model'

tokenizer = AlbertTokenizerFast.from_pretrained(albet_tokenizer_model)


file /content/drive/MyDrive/NOTAM/albert_tokenizer_model/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
file /content/drive/MyDrive/NOTAM/albert_tokenizer_model/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
op = tokenizer.encode("ais (xx) australian digital aeronautical flight information file edition 8 (ausdafif ed 8) withdrawn. replacement version australian digital aeronautical flight information file edition 8 version 2 (ausdafif ed 8 v2) available via ais-af intranet. mil use only.")
print(op)
tokenizer.decode(op)


[5, 2960, 158, 325, 150, 934, 121, 6471, 6891, 4465, 287, 5526, 227, 537, 784, 2603, 727, 158, 722, 13632, 11285, 124, 156, 482, 1503, 123, 3055, 203, 124, 9543, 934, 121, 6471, 6891, 4465, 287, 5526, 227, 537, 784, 2603, 727, 124, 9543, 244, 158, 722, 13632, 11285, 124, 156, 727, 309, 762, 542, 230, 2960, 133, 3530, 16070, 1695, 123, 493, 295, 217, 123, 6]


'[CLS] ais (xx) australian digital aeronautical flight information file edition 8 (ausdafif ed 8) withdrawn. replacement version australian digital aeronautical flight information file edition 8 version 2 (ausdafif ed 8 v2) available via ais-af intranet. mil use only.[SEP]'

In [21]:
# tokenizer에 special token 추가
special_tokens_dict = {'additional_special_tokens': user_defined_symbols}
tokenizer.add_special_tokens(special_tokens_dict)

# check tokenizer vocab with special tokens
print('check special tokens : %s'%tokenizer.all_special_tokens[:20])

check special tokens : ['[CLS]', '[SEP]', '<unk>', '<pad>', '[MASK]', '[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]']


In [22]:
# save tokenizer model with special tokens
tokenizer.save_pretrained(albet_tokenizer_model+'_special')

('/content/drive/MyDrive/NOTAM/albert_tokenizer_model_special/tokenizer_config.json',
 '/content/drive/MyDrive/NOTAM/albert_tokenizer_model_special/special_tokens_map.json',
 '/content/drive/MyDrive/NOTAM/albert_tokenizer_model_special/spiece.model',
 '/content/drive/MyDrive/NOTAM/albert_tokenizer_model_special/added_tokens.json',
 '/content/drive/MyDrive/NOTAM/albert_tokenizer_model_special/tokenizer.json')

In [23]:
from transformers import AlbertTokenizer, AlbertTokenizerFast

tokenizer = AlbertTokenizer.from_pretrained(albet_tokenizer_model+'_special')

In [24]:
op = tokenizer("ais (xx) australian digital aeronautical flight information file edition 8 (ausdafif ed 8) withdrawn. replacement version australian digital aeronautical flight information file edition 8 version 2 (ausdafif ed 8 v2) available via ais-af intranet. mil use only.", return_tensors="pt")
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))


{'input_ids': tensor([[    5,  2960,   158,   325,   150,   934,   121,  6471,  6891,  4465,
           287,  5526,   227,   537,   784,  2603,   727,   158,   722, 13632,
         11285,   124,   156,   482,  1503,   123,  3055,   203,   124,  9543,
           934,   121,  6471,  6891,  4465,   287,  5526,   227,   537,   784,
          2603,   727,   124,  9543,   244,   158,   722, 13632, 11285,   124,
           156,   727,   309,   762,   542,   230,  2960,   133,  3530, 16070,
          1695,   123,   493,   295,   217,   123,     6]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1

In [25]:
#Checking vocabulary size
vocab_size=tokenizer.vocab_size ; vocab_size

21770

In [26]:
import json

config = {
    "architectures": [
        "AlbertModel"
    ],
	"attention_probs_dropout_prob": 0.1,
	"hidden_act": "gelu",
	"hidden_dropout_prob": 0.1,
	"hidden_size": 768,
	"initializer_range": 0.02,
	"intermediate_size": 3072,
	"layer_norm_eps": 1e-05,
	"max_position_embeddings": 514,
	"model_type": "albert",
	"num_attention_heads": 12,
	"num_hidden_layers": 6,
	"type_vocab_size": 1,
	"vocab_size": vocab_size
}
with open(albet_tokenizer_model + "_special/config.json", 'w') as fp:
    json.dump(config, fp)


#Configuration for tokenizer.
#Note: I set do_lower_case: False, and keep_accents:True
# Opening JSON file
f = open(albet_tokenizer_model+ "_special/tokenizer_config.json")
   
# returns JSON object as 
# a dictionary
tokenizer_config = json.load(f)

tokenizer_config['max_len'] = 512
tokenizer_config['model_type'] = 'albert'
tokenizer_config['do_lower_case'] = False
tokenizer_config['keep_accents'] = True

with open(albet_tokenizer_model+ "_special/tokenizer_config.json", 'w') as outfile:
    json.dump(tokenizer_config, outfile)
'''
tokenizer_config = {
	"max_len": 512,
	"model_type": "albert",
	"do_lower_case":False, 
	"keep_accents":True
}
with open(albet_tokenizer_model+ "_special/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)
'''

'\ntokenizer_config = {\n\t"max_len": 512,\n\t"model_type": "albert",\n\t"do_lower_case":False, \n\t"keep_accents":True\n}\nwith open(albet_tokenizer_model+ "_special/tokenizer_config.json", \'w\') as fp:\n    json.dump(tokenizer_config, fp)\n'

In [ ]:

#To train from scratch
!python /content/transformers/examples/pytorch/language-modeling/run_mlm.py \
        --model_type albert-base-v2 \
        --config_name /content/drive/MyDrive/NOTAM/albert_tokenizer_model_special \
        --tokenizer_name /content/drive/MyDrive/NOTAM/albert_tokenizer_model_special \
        --train_file /content/drive/MyDrive/NOTAM/train_tokenizer.txt \
        --output_dir /content/drive/MyDrive/NOTAM/albert_tokenizer_model_special/model \
        --use_fast_tokenizer \
        --do_train \
        --line_by_line \
        --save_steps 500 \
        --logging_steps 500 \
        --save_total_limit 2 \
        --num_train_epochs 20 \
        --seed 108 \
        --overwrite_output_dir \
        --logging_dir /content/drive/MyDrive/Tokenizer_train/logs

2021-08-14 08:05:50.300670: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/14/2021 08:05:52 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/14/2021 08:05:52 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_

# language model 만들었으면... 여기서 부터~

In [17]:
#model_name = 'albert-base-v2'
model_name = '/content/drive/MyDrive/NOTAM/albert_tokenizer_model_special/model'

In [18]:
from transformers import AlbertTokenizer, AlbertTokenizerFast

tokenizer = AlbertTokenizer.from_pretrained(model_name)
op = tokenizer("ais (xx) australian digital aeronautical flight information file edition 8 (ausdafif ed 8) withdrawn. replacement version australian digital aeronautical flight information file edition 8 version 2 (ausdafif ed 8 v2) available via ais-af intranet. mil use only.", return_tensors="pt")
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))

{'input_ids': tensor([[   5, 3174,  157,  159,  151,  468,  121, 3250, 3367, 2698,  121,  304,
         2824,  345,  377,  842,  184, 2154, 1075,  157,  304, 4565, 5232,  126,
          158, 1075,  151, 1224,  127, 4404,  884,  126, 5033,  468,  121, 3250,
         3367, 2698,  121,  304, 2824,  345,  377,  842,  184, 2154, 1075,  126,
         5033,  260,  157,  304, 4565, 5232,  126,  158, 1075,  327,  706,  864,
          379, 3174,  140, 1271,  982,  378,  121,  614,  127,  296,  461,  182,
          127,    6]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1,

# Trainset, Testset

In [19]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split

In [28]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
labels = []
# For every sentence...
for row in df.iterrows():
    # E section
    sent = clean_text(str(row[1][11]).lower()) 
    # label for E Section
    lb = row[1][3]+0
    if lb == 6:
        lb = 5
    labels.append(lb)
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True,
                )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', notam_contents[0])
print('Token IDs:', input_ids[0])

# Training & Validation Split
# Divide up our training set to use 90% for training and 10% for validation.

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )   

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2187: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  ais (xx) australian digital aeronautical flight information file edition 8 (ausdafif ed 8) withdrawn. replacement version australian digital aeronautical flight information file edition 8 version 2 (ausdafif ed 8 v2) available via ais-af intranet. mil use only.
Token IDs: tensor([    5,  3174,   157,   159,   151,   468,   121,  3250,  3367,  2698,
          121,   304,  2824,   345,   377,   842,   184,  2154,  1075,   157,
          304,  4565,  5232,   126,   158,  1075,   151,  1224,   127,  4404,
          884,   126,  5033,   468,   121,  3250,  3367,  2698,   121,   304,
         2824,   345,   377,   842,   184,  2154,  1075,   126,  5033,   260,
          157,   304,  4565,  5232,   126,   158,  1075,   327,   706,   864,
          379,  3174,   140,  1271,   982,   378,   121,   614,   127,   296,
          461,   182,   127,     6, 11866, 11866, 11866, 11866, 11866, 11866,
        11866, 11866, 11866, 11866, 11866, 11866, 11866, 11866, 11866, 11866,
        11866,

# classification


In [29]:
from transformers import AlbertForSequenceClassification

model = AlbertForSequenceClassification.from_pretrained(
                                    model_name, # Use the 12-layer BERT model, with an uncased vocab.
                                    num_labels = 6, # The number of output labels--2 for binary classification.
                                                    # You can increase this for multi-class tasks.   
                                    output_attentions = False, # Whether the model returns attentions weights.
                                    output_hidden_states = False, # Whether the model returns all hidden-states.
                                )

Some weights of the model checkpoint at /content/drive/MyDrive/NOTAM/albert_tokenizer_model_special/model were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/NOTAM/albert_tok

In [22]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [30]:
import os

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

def save_notam_model(notam_model, output_dir = '/content/drive/MyDrive/NOTAM/notam_model'):
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Saving model to %s" % output_dir)

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = notam_model.module if hasattr(notam_model, 'module') else notam_model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

def load_notam_model(input_dir = '/content/drive/MyDrive/NOTAM/notam_model'):
    print('Loading Albert notam model...')
    tokenizer = AlbertTokenizer.from_pretrained(input_dir)
    model = AlbertForSequenceClassification.from_pretrained(input_dir)
    return tokenizer, model

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

In [39]:
from transformers import BertTokenizer, AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
import time
import random
import datetime
import numpy as np


epochs = 1

# Tell pytorch to run this model on the GPU.
model.cuda()

# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The Albert model has {:} different named parameters.\n'.format(len(params)))
 
print(model)

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
#epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
    
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        #print(b_input_ids)
        #print(b_input_mask)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.

        outputs =  model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels)
        '''
        loss, logits = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels)
        '''
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        loss = outputs[0]
        logits = outputs[1]
        
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    true_labels = []
    pred_labels = []
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        for l in batch[2]:
            true_labels.append(l.item())        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        for li in logits:
            pred_labels.append(np.argmax(li))
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    print("  classification_report   ")    
    print(classification_report(true_labels,pred_labels))
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    save_notam_model(model,output_dir='/content/drive/MyDrive/NOTAM/notam_model/For_E_Section')

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

The Albert model has 27 different named parameters.

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(11967, 128, padding_idx=0)
      (position_embeddings): Embedding(514, 128)
      (token_type_embeddings): Embedding(1, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
           

# Combining Categorical and Numerical Features with Text in BERT

https://mccormickml.com/2021/06/29/combining-categorical-numerical-features-with-bert/